In [121]:
import numpy as np
import pandas as pd
import math
import time
import pickle as pkl
import gensim
import shutil
import glob
import os
import smart_open
import time

In [62]:
IV_file = '../../data/inverted_index.pkl'
TAGDICFILE = '../../data/trainTagsToIndex.pkl'
DOC_LENGTH_FILE = '../../data/doc_length.pkl'
QUERIES_FILE = '../../data/queries.txt'
DOC2VECMODEL = '../../data/modelembedding.pkl'
stopwords = open('../code/stopwords.txt', 'r').readlines()
punctuation = open('../code/punctuation.txt', 'r').readlines()
stopwords = [i.strip() for i in stopwords]
punctuation = [i.strip() for i in punctuation]
f = open(IV_file, 'rb')
inverted_index = pkl.load(f)
f.close()
f = open(DOC_LENGTH_FILE, 'rb')
documentLenArr = pkl.load(f)
f.close()
f = open(TAGDICFILE, 'rb')
tagDic = pkl.load(f)
f.close()
numdocuments = len(documentLenArr)
k1 = 1.5
b = 0.5
avgDoclength = sum(documentLenArr)*1.0/numdocuments
maxranking = 50
f = open(DOC2VECMODEL, 'rb')
doc2vecmodel = pkl.load(f)
f.close()


In [59]:
def read_corpus_query(fname):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
#             print(i,line)
            tokens = gensim.utils.simple_preprocess(line)
            yield tokens
            


In [85]:
# assuming numdocuments, documentLenArr, k1, b, avgDoclength, maxrank
#usage sort_index(getScoreForQuery(find_count(query)))
def getScoreForQuery(queryMap):
    documentToScore = {}
    for term, termfreq in queryMap.items():
        if term in inverted_index:
            posting = inverted_index[term]
            weightOrIdf = calcIdf(len(posting))
            for docInfo in posting:
                doc_id = docInfo[0]
                doc_id_freq = docInfo[1]
                scoreTermDoc = getScoreForTermForDocument(termfreq,doc_id,weightOrIdf,doc_id_freq)
                if doc_id in documentToScore:
                    documentToScore[doc_id] = documentToScore[doc_id] + scoreTermDoc
                else:
                    documentToScore[doc_id] = scoreTermDoc
    return documentToScore
def getScoreForQueryOptimized(queryMap, qindex):
    documentToScore = {}
    thisdocset = docsets[qindex]
    for term, termfreq in queryMap.items():
        if term in inverted_index:
            posting = inverted_index[term]
            weightOrIdf = calcIdf(len(posting))
            for docInfo in posting:
                doc_id = docInfo[0]
                if doc_id not in thisdocset:
                    continue
                doc_id_freq = docInfo[1]
                scoreTermDoc = getScoreForTermForDocument(termfreq,doc_id,weightOrIdf,doc_id_freq)
                if doc_id in documentToScore:
                    documentToScore[doc_id] = documentToScore[doc_id] + scoreTermDoc
                else:
                    documentToScore[doc_id] = scoreTermDoc
    return documentToScore
def calcIdf(lenposting):
    ans = math.log((numdocuments*1.0 - lenposting +0.5)/(lenposting+0.5))
    return ans
def getScoreForTermForDocument(qfi,docNo,weight,tfi):
    docLength = documentLenArr[docNo]*1.0
    ans = weight* ((tfi*(k1+1)*1.0)/(tfi+k1*(1.0-b+b*(docLength/avgDoclength))))
    ans = ans*qfi
    return ans
def find_count(text_arr):
    dictionary = {}
    for i in text_arr:
        if (i not in stopwords and i not in punctuation):
            if (i in dictionary):
                dictionary[i] += 1
            else:
                dictionary[i] = 1
    return dictionary

In [42]:
def sort_index(documentToScore):
    ans = []
    numvaluesAdded=0
    while(numvaluesAdded<=maxranking and len(documentToScore)!=0):
        tempkey = max(documentToScore, key=documentToScore.get)
        ans.append((tempkey, documentToScore[tempkey]))
        numvaluesAdded +=1
        del documentToScore[tempkey]
    return ans
def tagToScore(documentToScore):
    ans = []
    for doc_id, score in documentToScore:
        ans.append((tagDic[doc_id], score))
    return ans;

In [113]:
def writeOutput(answers, filename):
    ansstr = ""
    for i in range(len(answers)):
        thisanswer = answers[i]
        doctagToScore = tagToScore(thisanswer)
        for j in range(len(doctagToScore)):
            tempstr = str(i+51) + " 0 "+ doctagToScore[j][0] + " " + str(j+1)+" "+ str(doctagToScore[j][1])+ " p\n"
#             print(tempstr)
            ansstr += tempstr
    outfile = open(filename, "w")
    outfile.write(ansstr)
    outfile.close()
    

In [114]:
writeOutput(answersall, "../../data/answers.txt")

In [119]:
writeOutput(answersallOptimized, "../../data/answersOptimized.txt")

In [92]:
%%time
answers0 = sort_index(getScoreForQuery(find_count(queries[0].split(' '))))

CPU times: user 2.25 s, sys: 6.93 ms, total: 2.26 s
Wall time: 2.25 s


In [91]:
%%time

answers0opti = sort_index(getScoreForQueryOptimized(find_count(queries[0].split(' ')),0))

CPU times: user 119 ms, sys: 2.06 ms, total: 122 ms
Wall time: 118 ms


In [93]:
answers0

[(26981, 304.8617475336065),
 (28810, 282.8851483024513),
 (53422, 274.5631892787238),
 (74614, 262.4044325005894),
 (1953, 245.41192241598677),
 (4555, 244.0334300231214),
 (68360, 231.43227164080542),
 (49438, 222.6534758597379),
 (36992, 212.04500435688288),
 (10610, 206.4714163471085),
 (69787, 199.4715704218965),
 (12481, 184.03169749426738),
 (75323, 177.86855546017236),
 (51054, 176.6515263676422),
 (61949, 173.10270791548734),
 (7363, 172.42968976118823),
 (48370, 169.97662687072503),
 (67497, 169.18133787243318),
 (10316, 166.49337042244358),
 (23817, 162.95225033705762),
 (69485, 162.73952222072364),
 (75202, 160.699891576353),
 (37326, 159.17784547817428),
 (55433, 153.89222430881756),
 (69523, 153.52976615585615),
 (74240, 152.26524742628152),
 (48369, 151.7036994552047),
 (79700, 151.2075064671312),
 (71967, 151.18797279251586),
 (48705, 151.01492318144201),
 (2351, 148.22282148041472),
 (2350, 148.09189543366784),
 (69450, 137.60360242023933),
 (72239, 136.43473572067273)

In [94]:
answers0opti

[(26981, 304.8617475336065),
 (28810, 282.8851483024513),
 (53422, 274.5631892787238),
 (74614, 262.4044325005894),
 (1953, 245.41192241598677),
 (4555, 244.0334300231214),
 (68360, 231.43227164080542),
 (49438, 222.6534758597379),
 (75323, 177.86855546017236),
 (61949, 173.10270791548734),
 (7363, 172.42968976118823),
 (48370, 169.97662687072503),
 (10316, 166.49337042244358),
 (23817, 162.95225033705762),
 (69485, 162.73952222072364),
 (75202, 160.699891576353),
 (37326, 159.17784547817428),
 (69523, 153.52976615585615),
 (74240, 152.26524742628152),
 (79700, 151.2075064671312),
 (48705, 151.01492318144201),
 (69450, 137.60360242023933),
 (80574, 131.72363636114514),
 (7617, 129.4138074700494),
 (69466, 128.51357776581312),
 (37510, 126.64943764383376),
 (19165, 125.4472371871416),
 (3352, 122.49380693124816),
 (37411, 120.90206213948221),
 (45366, 119.28975015452562),
 (81775, 118.35800388105268),
 (20570, 113.63497484506628),
 (68622, 112.07333054710834),
 (29976, 110.8909967847756

In [125]:
%%time
starttime = time.time()
queries = open(QUERIES_FILE, 'r').readlines()
answersall = []
for x in range(len(queries)):
    answersall.append(sort_index(getScoreForQuery(find_count(queries[x].split(' ')))))
endtime = time.time()

CPU times: user 1min 31s, sys: 56.3 ms, total: 1min 31s
Wall time: 1min 31s


In [126]:
print(endtime-starttime)

91.34549808502197


In [43]:
tagToScore(answers0)

[('AP880412-0268', 304.8617475336065),
 ('AP880406-0267', 282.8851483024513),
 ('AP880316-0292', 274.5631892787238),
 ('AP880301-0271', 262.4044325005894),
 ('AP880731-0085', 245.41192241598677),
 ('AP880325-0293', 244.0334300231214),
 ('AP890608-0253', 231.43227164080542),
 ('AP880922-0206', 222.6534758597379),
 ('AP880512-0314', 212.04500435688288),
 ('AP891010-0258', 206.4714163471085),
 ('AP880503-0244', 199.4715704218965),
 ('AP890329-0221', 184.03169749426738),
 ('AP890216-0288', 177.86855546017236),
 ('AP890606-0198', 176.6515263676422),
 ('AP881017-0296', 173.10270791548734),
 ('AP890412-0286', 172.42968976118823),
 ('AP890419-0228', 169.97662687072503),
 ('AP891011-0255', 169.18133787243318),
 ('AP890503-0279', 166.49337042244358),
 ('AP881219-0271', 162.95225033705762),
 ('AP880626-0038', 162.73952222072364),
 ('AP890216-0167', 160.699891576353),
 ('AP890816-0062', 159.17784547817428),
 ('AP890404-0263', 153.89222430881756),
 ('AP880626-0076', 153.52976615585615),
 ('AP880926

In [123]:
%%time
starttime = time.time()
queries = open(QUERIES_FILE, 'r').readlines()
test_corpus = list(read_corpus_query(QUERIES_FILE))
docsets = []
for doc_id in range(len(test_corpus)):
    inferred_vector = doc2vecmodel.infer_vector(test_corpus[doc_id])
    sims = doc2vecmodel.docvecs.most_similar([inferred_vector], topn=5000)
#     print(sims[:])
    docsets.append(set([x for x,_ in sims]))
answersallOptimized = []
for x in range(len(queries)):
    answersallOptimized.append(sort_index(getScoreForQueryOptimized(find_count(queries[x].split(' ')),x)))
endtime = time.time()

CPU times: user 12.9 s, sys: 47.9 ms, total: 13 s
Wall time: 13.9 s


In [124]:
print(endtime-starttime)

13.852821111679077


In [99]:
tagToScore(answersallOptimized[0])

[('AP880412-0268', 304.8617475336065),
 ('AP880406-0267', 282.8851483024513),
 ('AP880316-0292', 274.5631892787238),
 ('AP880301-0271', 262.4044325005894),
 ('AP880731-0085', 245.41192241598677),
 ('AP880325-0293', 244.0334300231214),
 ('AP890608-0253', 231.43227164080542),
 ('AP880922-0206', 222.6534758597379),
 ('AP880512-0314', 212.04500435688288),
 ('AP891010-0258', 206.4714163471085),
 ('AP880503-0244', 199.4715704218965),
 ('AP890329-0221', 184.03169749426738),
 ('AP890216-0288', 177.86855546017236),
 ('AP890606-0198', 176.6515263676422),
 ('AP881017-0296', 173.10270791548734),
 ('AP890412-0286', 172.42968976118823),
 ('AP890419-0228', 169.97662687072503),
 ('AP891011-0255', 169.18133787243318),
 ('AP890503-0279', 166.49337042244358),
 ('AP881219-0271', 162.95225033705762),
 ('AP880626-0038', 162.73952222072364),
 ('AP890216-0167', 160.699891576353),
 ('AP890816-0062', 159.17784547817428),
 ('AP890404-0263', 153.89222430881756),
 ('AP880626-0076', 153.52976615585615),
 ('AP880926

In [44]:
avgDoclength

245.3014302101384

In [81]:
26981 in allsims[0]

True

In [83]:
[(x[0] in allsims[0]) for x in answers0]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False]